In [1]:
import pandas as pd

passengers = pd.read_csv("Data/train.csv")
passengers.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
x = passengers.drop(columns=["Survived"])
y = passengers["Survived"]

In [3]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin


class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        
        self.age_bins = [0, 3, 12, 20, 60, 200]
        self.age_labels = ["toddler", "child", "teen", "adult", "senior"]
        
        self.q25 = 0
        self.q50 = 0
        self.q75 = 0
        self.fare_bins = []
        self.fare_labels = ["VeryLow", "Low", "Medium", "High"]
                
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        
        # Extracting the title
        X["Title"] = X["Name"].apply(lambda x: x.split(", ")[1].split(".")[0])
        # Grouping the titles
        X['Title'] = X['Title'].apply(lambda x: x if x in ["Mr", "Miss", "Mrs", "Master"] else "Rare")
        X['IsTitleRare'] = X['Title'].apply(lambda x: 1 if x=="Rare" else 0)
        
        # Defining a new feature - age group
        X['Age'].fillna(X['Age'].median(), inplace=True)
        X["AgeGroup"] = pd.cut(X["Age"], bins=self.age_bins, labels=self.age_labels)
        
        # Person + family members on board
        X["FamilySize"] = X["SibSp"] + X["Parch"] + 1
        
        # Calculate adjusted fare (ticket price per person)
        X['Fare'].fillna(X['Fare'].median(), inplace=True)
        X["TicketCount"] = X["Ticket"].map(X["Ticket"].value_counts())
        X["AdjustedFare"] = round(X["Fare"] / X["TicketCount"], 4)
        
        # Replacing Fares = 0 with median and adding log fare
        X['AdjustedFare'].replace(0, X['AdjustedFare'].median(), inplace=True)
        X['LogFare'] = np.log(X['AdjustedFare'])
        
        # Extracting wealth from fare
        self.q25, self.q50, self.q75 = list(X['AdjustedFare'].quantile([.25, .5, .75]))
        self.fare_bins = [0, self.q25, self.q50, self.q75, X['AdjustedFare'].max()]
        X["Wealth"] = pd.cut(X["AdjustedFare"], bins=self.fare_bins, labels=self.fare_labels)
        
        # Extracting deck of residance for each passanger
        X["Deck"] = X["Cabin"].str[0]
        X["Deck"].fillna("N/A", inplace=True)
        
        # Check if passenger has cabin
        X['HasCabin'] = X['Deck'].apply(lambda x: 0 if x=="N/A" else 1)
        
        # Check if passenger is traveling solo
        X['IsAlone'] = X['FamilySize'].apply(lambda x: 1 if x==1 else 0)

        
        return X

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


num_features = ['Age', 'LogFare', 'FamilySize']
cat_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'Deck']
drop_features = ['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessing = Pipeline([
    ('feature_engineering', FeatureEngineering()),
    ('preprocessor', ColumnTransformer([
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features),
        ('dropper', 'drop', drop_features)
    ]))
])

In [ ]:
x_preproc = preprocessing.fit_transform(x)

['Age',
 'LogFare',
 'FamilySize',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'Title_Master',
 'Title_Miss',
 'Title_Mr',
 'Title_Mrs',
 'Title_Rare',
 'Deck_A',
 'Deck_B',
 'Deck_C',
 'Deck_D',
 'Deck_E',
 'Deck_F',
 'Deck_G',
 'Deck_N/A',
 'Deck_T']